Data Science - Final project

By Gal Levi and Liel Cohen

Crawling

In [1]:
import re
import time
from datetime import datetime

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
class LaptopCrawler:
    def __init__(self, chromedriver_path, url):
        self.chromedriver_path = chromedriver_path
        self.url = url
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def get_laptops(self):
        laptops = []
        self.driver.get(self.url)
        for i in range(160):
            elements = self.driver.find_elements(By.CLASS_NAME, "more-details")

            # Retrieve the href attribute of each element
            for element in elements:
                href = element.get_attribute("href")
                laptops.append(href)
            self.driver.find_element(by=By.CLASS_NAME, value="page-item.next").click()
        return laptops

    def get_details(self, laptops):
        all_details = []
        for laptop in laptops:
            time.sleep(2)
            self.driver.get(laptop)
            crawling_time = datetime.now()
            price_and_stores = self.driver.find_element(By.CLASS_NAME, "prices-txt").text
            price_and_stores = re.findall(r'\b\d+(?:,\d+)?\b', price_and_stores)
            laptop_val = {}
            rows_values = self.driver.find_elements(By.CLASS_NAME, "paramRow")
            for row in rows_values:
                param_and_value = row.text.split('\n')
                if len(param_and_value) == 2:
                    laptop_val[param_and_value[0]] = param_and_value[1]
            if len(price_and_stores) == 3:
                laptop_val['lower_price'] = price_and_stores[0]
                laptop_val['higher_price'] = price_and_stores[1]
                laptop_val['num_of_stores'] = price_and_stores[2]
            elif len(price_and_stores) == 2:
                laptop_val['lower_price'] = price_and_stores[0]
                laptop_val['higher_price'] = price_and_stores[0]
                laptop_val['num_of_stores'] = price_and_stores[1]
            else:
                laptop_val['lower_price'] = price_and_stores[0]
                laptop_val['higher_price'] = price_and_stores[0]
                laptop_val['num_of_stores'] = 1
            laptop_val['crawling_time'] = crawling_time
            all_details.append(laptop_val)
        df = pd.DataFrame(all_details)
        df.to_excel("output.xlsx")

In [ ]:
class BenchmarkCrawler:
    def __init__(self, chromedriver_path, url):
        self.chromedriver_path = chromedriver_path
        self.url = url
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def crawl_cpu(self):
        # Load the web page
        self.driver.get(self.url)

        # Find the table element by ID
        table = self.driver.find_element(By.ID, "cputable")

        # Find all rows in the table
        rows = table.find_elements(By.TAG_NAME, "tr")

        # Create empty lists to store the column data
        cpu_names = []
        cpu_marks = []
        ranks = []
        prices = []

        # Loop through the rows and extract the data
        for row in rows[1:]:  # Exclude the header row
            # Find columns within each row
            columns = row.find_elements(By.TAG_NAME, "td")

            # Extract the text from each column
            cpu_name = columns[0].text
            cpu_mark = columns[1].text
            rank = columns[2].text
            price = columns[3].text

            # Append the data to the respective lists
            cpu_names.append(cpu_name)
            cpu_marks.append(cpu_mark)
            ranks.append(rank)
            prices.append(price)

        # Close the WebDriver
        self.driver.quit()

        # Create a DataFrame from the data lists
        data = {"CPU_name": cpu_names, "CPU_mark": cpu_marks, "Rank": ranks, "Price": prices}
        df = pd.DataFrame(data)

        df.to_excel("benchmark.xlsx")

In [ ]:
def main():
    chromedriver_path = r"../chromedriver.exe"
    url = "https://www.zap.co.il/models.aspx?sog=c-pclaptop"
    cpu_banchmark_crawler = BenchmarkCrawler(chromedriver_path, 'https://www.cpubenchmark.net/cpu_list.php')
    cpu_banchmark_crawler.crawl_cpu()

    laptop_crawler = LaptopCrawler(chromedriver_path, url)
    laptops = laptop_crawler.get_laptops()
    laptop_crawler.get_details(laptops)


if __name__ == '__main__':
    main()

Crosscheck CPU benchmark

In [ ]:
# Read the input files
df1 = pd.read_excel('output.xlsx', sheet_name='Sheet1', index_col=None)
df1 = df1.drop(columns=df1.columns[0])
df2 = pd.read_excel('benchmark.xlsx', sheet_name='Sheet1', index_col=None)
df2 = df2.drop(columns=df2.columns[0])

# Create a copy of the DataFrames
data = pd.DataFrame.copy(df1)
cpu_benchmark = pd.DataFrame.copy(df2)


# Function to get the CPU mark based on the processor model and type
def get_cpu_mark(row):
    processor_model = row['דגם מעבד']
    processor_type = row['סוג מעבד']
    matched_row = cpu_benchmark[cpu_benchmark['CPU_name'].str.contains(processor_model, case=False)]
    if not matched_row.empty:
        return matched_row['CPU_mark'].values[0]
    matched_row = cpu_benchmark[cpu_benchmark['CPU_name'].str.contains('Apple ' + processor_type, case=False)]
    if not matched_row.empty:
        return matched_row['CPU_mark'].values[0]
    else:
        return None


# Add CPU_mark column to data using apply function
data['CPU_mark'] = data.apply(lambda row: get_cpu_mark(row), axis=1)
data.to_excel("all_details.xlsx", index=False)

Data Cleaning

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
# Copy df
df1 = pd.read_excel('all_details.xlsx', sheet_name='Sheet1', index_col=None)
data = pd.DataFrame.copy(df1)

In [ ]:
# Calculate the average price
data['lower_price'] = data['lower_price'].str.replace(',', '').astype(int)
data['higher_price'] = data['higher_price'].str.replace(',', '').astype(int)
data['average_price'] = (data['lower_price'] + data['higher_price']) / 2

# Drop irrelevant columns
cols_to_drop = ['סדרה', 'סוג מעבד', 'דגם מעבד', 'סוג הזכרון', 'סוג כונן קשיח',
                'כונן אופטי', 'רזולוציית מסך', 'סוג מסך', 'מצלמת רשת', 'כרטיס מסך',
                'תצורת 2 in 1', 'רשת אלחוטית', 'מודם סלולארי', 'crawling_time', 'lower_price', 'higher_price']
data = data.drop(cols_to_drop, axis=1)

# Change the col names to english
column_mapping = {
    'יצרן': 'manufacturer',
    'תאריך כניסה לזאפ': 'date_of_entry_to_Zap',
    'התאמה לגיימינג': 'gaming_Compatibility',
    'מערכת הפעלה': 'operating_system',
    'משקל': 'weight',
    'נפח זיכרון RAM': 'RAM_capacity',
    'מהירות מעבד': 'CPU_speed',
    'דור מעבד': 'CPU_generation',
    'נפח אחסון': 'storage_capacity',
    'גודל מסך': 'screen_size',
    'קצב רענון תצוגה': 'refresh_rate',
    'מסך מגע': 'touch_screen',
    'אמצעי אבטחה': 'security_features',
    'חיבורים': 'connections'
}
data.rename(columns=column_mapping, inplace=True)

In [ ]:
# check the data
data.info()
print(data.describe(include='all'))
print(data)
print(data.isnull().sum())
print(data.duplicated(subset=None).sum())

In [ ]:
# convert objects to float
data['weight'] = data['weight'].apply(
    lambda x: float(re.findall(r'\d+(?:\.\d+)?', x)[0]) if re.findall(r'\d+(?:\.\d+)?', x) else None)
data['CPU_speed'] = data['CPU_speed'].apply(
    lambda x: float(re.findall(r'\d+(?:\.\d+)?', x)[0]) if re.findall(r'\d+(?:\.\d+)?', x) else None)
data['screen_size'] = data['screen_size'].apply(
    lambda x: float(re.findall(r'\d+(?:\.\d+)?', x)[0]) if re.findall(r'\d+(?:\.\d+)?', x) else None)

# convert objects to int
data['RAM_capacity'] = data['RAM_capacity'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)
data['RAM_capacity'] = data['RAM_capacity'].astype('Int64')
data['CPU_generation'] = data['CPU_generation'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)
data['CPU_generation'] = data['CPU_generation'].astype('Int64')
data['storage_capacity'] = data['storage_capacity'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)
data['storage_capacity'] = data['storage_capacity'].astype('Int64')
data['refresh_rate'] = data['refresh_rate'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)
data['refresh_rate'] = data['refresh_rate'].astype('Int64')
data['date_of_entry_to_Zap'] = data['date_of_entry_to_Zap'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else None)
data['date_of_entry_to_Zap'] = data['date_of_entry_to_Zap'].astype('Int64')
data['CPU_mark'] = data['CPU_mark'].fillna('0').str.replace(',', '').astype('int64')
data['connections'] = [len(row.split(', ')) for row in data['connections']]

# convert objects to categorical
data['gaming_Compatibility'] = [1 if x == 'גיימינג' else 0 for x in data['gaming_Compatibility']]
data['gaming_Compatibility'] = data['gaming_Compatibility'].astype('category')
data['touch_screen'] = [1 if x in ['כולל', 'מסך מגע משני'] else 0 for x in data['touch_screen']]
data['touch_screen'] = data['touch_screen'].astype('category')
data['operating_system'] = data['operating_system'].apply(lambda x: 1 if x != 'ללא' else 0)
data['operating_system'] = data['operating_system'].astype('category')
data['security_features'] = [
    0 if x in ['לא כולל', 'לא רלוונטי', 'לא זמין'] else 1 for x in data['security_features']]
data['security_features'] = data['security_features'].astype('category')

In [ ]:
# Remove outliers
new_df = data.copy()
numeric_cols = new_df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    z_score = (data[col] - data[col].mean()) / data[col].std()
    outliers = abs(z_score) > 3
    new_df.loc[outliers, col] = np.nan

data_no_outliers = new_df

In [ ]:
# check the data again
print(data_no_outliers.isnull().sum())
data_no_outliers.dropna(inplace=True)
print(data_no_outliers.isnull().sum())
print(data_no_outliers.duplicated(subset=None).sum())
data_no_outliers = data_no_outliers.drop_duplicates()
print(data_no_outliers.duplicated(subset=None).sum())
data_no_outliers.to_excel("all_details_cleaned.xlsx", index=False)

EDA and visualisition

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the DataFrame
df = pd.read_excel('all_details_cleaned.xlsx', sheet_name='Sheet1', index_col=None)

In [ ]:
# Histograms
df.hist(figsize=(10, 8))
plt.tight_layout()
plt.show()

In [ ]:
# Bar plots
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x="manufacturer")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='manufacturer', y='average_price', data=df)
plt.xlabel('Manufacturer')
plt.ylabel('Average Price')
plt.title('Average Price by Manufacturer')
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.show()

In [ ]:
# Heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(df.corr(), annot=True)
plt.show()

In [ ]:
# Line plot: Price over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x="date_of_entry_to_Zap", y="average_price", hue="manufacturer")
plt.xlabel("Date of Entry to Zap")
plt.ylabel("Average Price")
plt.title("Price Over Time by Manufacturer")
plt.xticks(rotation=45)
plt.legend(loc="best")
plt.show()

Machine Learning

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Load the data
df = pd.read_excel('all_details_cleaned.xlsx', sheet_name='Sheet1', index_col=None)

# Separate features and target variable
X = df.drop(["average_price", "manufacturer"], axis=1)  # Features
y = df["average_price"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_predictions = lr.predict(X_test_scaled)
lr_r2 = r2_score(y_test, lr_predictions)
mse = mean_squared_error(y_test, lr_predictions)
print("Linear Regression results:")
print("R^2 Score:", lr_r2)
print("Mean Squared Error (MSE):", mse)

# First
# Calculate the difference between predicted and actual values
residuals = lr_predictions - y_test

# Assign colors to point based on the difference
colors = np.where(residuals >= 0, 'g', 'r')

# Create scatter plot of predicted vs actual values with colors
plt.scatter(y_test, lr_predictions, c=colors)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Linear Regression: Actual vs Predicted Prices")
# Add the linear regression line with colors
plt.plot(y_test, y_test, color='blue', linewidth=3)
plt.fill_between(y_test, y_test, lr_predictions, where=(lr_predictions >= y_test), facecolor='green', alpha=0.3)
plt.fill_between(y_test, y_test, lr_predictions, where=(lr_predictions == y_test), facecolor='pink', alpha=0.3)
plt.fill_between(y_test, y_test, lr_predictions, where=(lr_predictions < y_test), facecolor='red', alpha=0.3)
plt.show()

# second
# Create scatter plot of predicted vs actual values
plt.scatter(y_test, lr_predictions, color='black')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Linear Regression: Actual vs Predicted Prices")

# Add the linear regression line
plt.plot(y_test, y_test, color='blue', linewidth=3)
plt.show()

# Third
# Create scatter plot of predicted vs actual values
plt.scatter(y_test, lr_predictions, color='lightblue', label='Predicted')
plt.scatter(y_test, y_test, color='lightgreen', label='Actual')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Linear Regression: Actual vs Predicted Prices")
plt.legend()

# Add the linear regression line
plt.plot(y_test, y_test, color='blue', linewidth=3)
plt.show()

In [ ]:
# k-Nearest Neighbors (k-NN)
knn = KNeighborsRegressor(n_neighbors=11)
knn.fit(X_train_scaled, y_train)
knn_predictions = knn.predict(X_test_scaled)
knn_r2 = r2_score(y_test, knn_predictions)
mse = mean_squared_error(y_test, knn_predictions)
print("k-NN results:")
print("R^2 Score:", knn_r2)
print("Mean Squared Error (MSE):", mse)

In [ ]:
# Decision Tree
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
dt_predictions = dt.predict(X_test)
dt_r2 = r2_score(y_test, dt_predictions)
mse = mean_squared_error(y_test, dt_predictions)
print("Decision Tree results:")
print("R^2 Score:", dt_r2)
print("Mean Squared Error (MSE):", mse)

In [ ]:
# Random Forest Regression
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
rf_r2 = r2_score(y_test, rf_predictions)
mse = mean_squared_error(y_test, rf_predictions)
print("Random Forest results:")
print("R^2 Score:", rf_r2)
print("Mean Squared Error (MSE):", mse)


# Plot actual vs predicted values
plt.scatter(y_test, rf_predictions, alpha=0.5)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values (Random Forest Regression)")
plt.show()

# Plot actual vs predicted values with colors
plt.scatter(y_test, rf_predictions, c=y_test, cmap='coolwarm', alpha=0.5)
plt.colorbar()
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values (Random Forest Regression)")
plt.show()